In [1]:
import pandas as pd
import utils.ediphi as ediphi

db = ediphi.Database()

def slow_describe_csi_sorts(data):
    with open('./queries/codes_and_descriptions_mf.sql', 'r') as m:
        mf = db.query(query=m.read())
    with open('./queries/codes_and_descriptions_uf.sql', 'r') as u:
        uf = db.query(query=u.read())
    for i in range(1,4):
        for n in range(len(data)):
            data[n].update(next(({f'mf{i}_desc': item[f'mf{i}_desc']} for item in mf if item[f'mf{i}_code'] == data[n][f'mf{i}_code']), ''))
            data[n].update(next(({f'uf{i}_desc': item[f'uf{i}_desc']} for item in uf if item[f'uf{i}_code'] == data[n][f'uf{i}_code']),''))
    return data

def describe_csi_sorts(df):
    with open('./queries/codes_and_descriptions_mf.sql', 'r') as m:
        df_mf = db.query(query=m.read(), df=True)
    with open('./queries/codes_and_descriptions_uf.sql', 'r') as u:
        df_uf = db.query(query=u.read(), df=True)
    for n in range(1,4):
        df_mf_l = df_mf[[f'mf{n}_code', f'mf{n}_desc']].drop_duplicates()
        df_uf_l = df_uf[[f'uf{n}_code', f'uf{n}_desc']].drop_duplicates()
        df = df.merge(df_mf_l, on=f'mf{n}_code', how='left')
        df = df.merge(df_uf_l, on=f'uf{n}_code', how='left')
    cols = ['id', 'name', 'uom'] +\
        [item for sublist in [(f'mf{n}_code', f'mf{n}_desc') for n in range(1,4)] for item in sublist] +\
        [item for sublist in [(f'uf{n}_code', f'uf{n}_desc') for n in range(1,4)] for item in sublist]
    return df[cols]

def get_custom_sorts(df, target='upc', estimate_id=None, sorts=None):
    if all([(target!='upc'), (estimate_id is None)]):
        raise ValueError('enter an estimate_id')
    file = './queries/sorts_upc.sql' if target == 'upc' else './queries/sorts_estimate.sql'
    with open(file, 'r') as q:
        df_cs = db.query(query=q.read().replace('__ESTIMATE_ID__',  str(estimate_id)), df=True)
    sorts = sorts if sorts else df_cs['code_name'].drop_duplicates().to_list()
    cols = list(df.columns)
    for sort in sorts:
        df_cs_l = df_cs.loc[df_cs['code_name'] == sort, ['id', 'code', 'description']].drop_duplicates()
        df_cs_l.columns = ['id', f'{sort}_code', f'{sort}_desc']
        df = df.merge(df_cs_l, on='id', how='left')
        cols+=[f'{sort}_code', f'{sort}_desc']
    return df[cols]

In [129]:
with open('./queries/base_upc.sql', 'r') as q:
    df = db.query(query=q.read(), df=True)

df.head(1)

,mf3_code,uf2_code,mf1_code,uf3_code,id,uf1_code,name,uom,mf2_code
0,01 00 00,None,01 00 00,None,fe830df5-a690-4430-b932-4bfc7e95f642,None,Subcontract - Construction Surveying,ls,01 00 00


In [132]:
with open('./queries/base_upc.sql', 'r') as q:
    data = db.query(query=q.read())

data[1]

{'mf3_code': '01 00 00',
 'uf2_code': None,
 'mf1_code': '01 00 00',
 'uf3_code': None,
 'id': 'fe809d2a-7678-4f48-90cb-ebc6fab13327',
 'uf1_code': None,
 'name': 'Subcontract - General Trades',
 'uom': 'ls',
 'mf2_code': '01 00 00'}

In [131]:
%timeit describe_csi_sorts(df)

1.17 s ± 22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [133]:
%timeit slow_describe_csi_sorts(data)

10.7 s ± 187 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
estimate_id = 'b5790ff4-1edb-49cc-a529-23d4401e24de'
with open('./queries/base_estimate_lines.sql', 'r') as q:
    df_el = db.query(query=q.read().replace('__ESTIMATE_ID__',  str(estimate_id)), df=True)

df_el = df_el[['id', 'name', 'uom', 'mf1_code', 'mf2_code', 'mf3_code', 'uf1_code', 'uf2_code', 'uf3_code']]
display(df_el.head())

,id,name,uom,mf1_code,mf2_code,mf3_code,uf1_code,uf2_code,uf3_code
0,99cacb82-77ce-4409-ae68-b8de7bd9a6ac,SEE SPEC 09 96 00 FOR HIGH PERFORMANCE COATINGS,None,05 00 00,05 51 00,05 51 00,None,None,None
1,3777d490-3f49-4d7e-9bc2-da73f2251633,"Concrete Misc Wall - 10"" Two-Sided",sf,03 00 00,03 30 00,03 30 00,B,B10,B1010
2,1c632869-e3f3-4d7a-bf48-a2986b9616c8,SEE SPEC 05 73 13 FOR ORNAMENTAL GLASS RAILINGS,None,05 00 00,05 51 00,05 51 00,None,None,None
3,a8e10bea-ced8-4e42-93bc-49f6ad611338,SEE SPEC 05 52 00 FOR METAL RAILINGS,None,05 00 00,05 51 00,05 51 00,None,None,None
4,e785c152-d89b-46c1-8462-cefa9fb4c6af,"Concrete Misc Wall - 12"" Two-Sided",sf,03 00 00,03 30 00,03 30 00,B,B10,B1010


In [76]:
df_res2 = get_custom_sorts(df_el, target='estimate', estimate_id=estimate_id, sorts=['Bid Package',])
display(df_res2.head())

,id,name,uom,mf1_code,mf2_code,mf3_code,uf1_code,uf2_code,uf3_code,Bid Package_code,Bid Package_desc
0,99cacb82-77ce-4409-ae68-b8de7bd9a6ac,SEE SPEC 09 96 00 FOR HIGH PERFORMANCE COATINGS,None,05 00 00,05 51 00,05 51 00,None,None,None,05.15,Structural Steel (Install)
1,3777d490-3f49-4d7e-9bc2-da73f2251633,"Concrete Misc Wall - 10"" Two-Sided",sf,03 00 00,03 30 00,03 30 00,B,B10,B1010,03.30,Cast-In-Place Concrete
2,1c632869-e3f3-4d7a-bf48-a2986b9616c8,SEE SPEC 05 73 13 FOR ORNAMENTAL GLASS RAILINGS,None,05 00 00,05 51 00,05 51 00,None,None,None,05.15,Structural Steel (Install)
3,a8e10bea-ced8-4e42-93bc-49f6ad611338,SEE SPEC 05 52 00 FOR METAL RAILINGS,None,05 00 00,05 51 00,05 51 00,None,None,None,05.15,Structural Steel (Install)
4,e785c152-d89b-46c1-8462-cefa9fb4c6af,"Concrete Misc Wall - 12"" Two-Sided",sf,03 00 00,03 30 00,03 30 00,B,B10,B1010,03.30,Cast-In-Place Concrete


In [3]:
def paginated_query(
        self,
        query: str,
        chunk_limit: int = 1000,
        pk: str = "id",
        df: bool = False,
    ):
        init_query = query + f' order by {pk} asc limit {chunk_limit}'
        res, result, idx = (1,1,0)
        try:
            init_query = query + f' order by {pk} asc limit {chunk_limit}'
            res = self.query(init_query)
            while (len(result) > 0) & (idx < 100_000):
                iter_query = query + f" where {pk} > '{res[-1][pk]}' order by {pk} asc limit {chunk_limit}"
                result = self.query(iter_query)
                res += result
                idx += 1
        except Exception as e:
            raise ValueError(e)
        if df:
            return pd.DataFrame(res)
        else:
            return res

In [ ]:
with open('./queries/base_estimate_lines.sql', 'r')
df = paginated_query(db, )